# Asistente Turístico de Tenerife
**Práctica Final - Large Language Models**

In [ ]:
# Section 1: Setup
from src.conf import OPENAI_API_KEY, MODEL_CONFIG, PDF_PATH
from src.api_client import OpenAIClient
from src.data_loader import DataLoader

print(f"API Key loaded: {OPENAI_API_KEY[:8]}...")
print(f"Model config: {MODEL_CONFIG}")

In [ ]:
# Initialize OpenAI client
client = OpenAIClient()
print("✅ OpenAI connection:", "OK" if client.test_connection() else "FAILED")

In [ ]:
# Load PDF and build vector store
from src.vector_store import VectorStore

loader = DataLoader(PDF_PATH)
pages = loader.load()
print(f"✅ PDF loaded: {loader.get_stats()}")

vector_store = VectorStore()
num_chunks = vector_store.build_from_documents(pages)
print(f"✅ Vector store built: {num_chunks} chunks")

In [ ]:
# Initialize Weather Service and RAG Chain with function calling
from src.weather_service import WeatherService
from src.rag_chain import RAGChain

weather_service = WeatherService(simulated=True)
rag = RAGChain(client, vector_store, weather_service=weather_service, max_history=5)

print("✅ Weather Service initialized (simulated mode)")
print("✅ RAG Chain initialized with function calling")

In [ ]:
# Test 1: Regular RAG query (no function call)
print("=" * 60)
print("TEST 1: Regular RAG Query")
print("=" * 60)

result = rag.query("¿Qué puedo visitar en La Laguna?", k=5)
print(f"👤 Usuario: {result['question']}")
print(f"\n🤖 Asistente: {result['answer']}")
print(f"\n🔧 Tool called: {result.get('tool_called', False)}")

In [ ]:
# Test 2: Weather query (should trigger function call)
print("=" * 60)
print("TEST 2: Weather Query (Function Calling)")
print("=" * 60)

result = rag.query("¿Qué tiempo hará mañana en Tenerife?")
print(f"👤 Usuario: {result['question']}")
print(f"\n🤖 Asistente: {result['answer']}")
print(f"\n🔧 Tool called: {result.get('tool_called', False)}")

In [7]:
# Test 3: Another weather query with specific date
print("=" * 60)
print("TEST 3: Specific Date Weather Query")
print("=" * 60)

from datetime import datetime, timedelta
future_date = (datetime.now() + timedelta(days=3)).strftime("%Y-%m-%d")

result = rag.query(f"¿Cómo estará el clima el {future_date}? Quiero ir a la playa.")
print(f"👤 Usuario: {result['question']}")
print(f"\n🤖 Asistente: {result['answer']}")
print(f"\n🔧 Tool called: {result.get('tool_called', False)}")

TEST 3: Specific Date Weather Query


2026-01-21 21:51:54,756 - INFO - Search query: '¿Cómo estará el clima el 2026-01-24? Quiero ir a l...' returned 3 results
2026-01-21 21:51:54,758 - INFO - Retrieved 3 chunks for: ¿Cómo estará el clima el 2026-01-24? Quiero ir a l...
2026-01-21 21:51:57,050 - INFO - Completion with tools - finish_reason: tool_calls
2026-01-21 21:51:57,051 - INFO - Tool call requested: get_weather
2026-01-21 21:51:57,051 - INFO - Weather request: date=2026-01-24, location=Tenerife, simulated=True
2026-01-21 21:51:57,052 - INFO - Weather response: Parcialmente nublado, 18°C
2026-01-21 21:52:01,023 - INFO - Completion received - tokens used: 1191
2026-01-21 21:52:01,024 - INFO - Generated answer after tool call: 594 chars


👤 Usuario: ¿Cómo estará el clima el 2026-01-24? Quiero ir a la playa.

🤖 Asistente: El clima para el 24 de enero de 2026 en Tenerife se prevé de la siguiente manera:

- **Temperatura máxima**: 18°C
- **Temperatura mínima**: 14°C
- **Condición**: Parcialmente nublado
- **Humedad**: 58%
- **Velocidad del viento**: 11 km/h

¡Parece un buen día para disfrutar de la playa! Te recomendaría llevar gafas de sol y, si decides visitar alguna playa, la **Playa de Benijo** podría ser una excelente opción, especialmente si te gusta ver atardeceres espectaculares. 

Recuerda que, aunque el clima es favorable, siempre es bueno estar preparado. ¡Espero que disfrutes tu día en la playa!

🔧 Tool called: True


In [ ]:
# Test 4: Multi-turn conversation
print("=" * 60)
print("TEST 4: Multi-turn Conversation")
print("=" * 60)

rag.clear_history()

In [ ]:
# Turn 1
result = rag.query("¿Qué playas hay en el norte de Tenerife?", k=5)
print(f"👤 Usuario: {result['question']}")
print(f"🤖 Asistente: {result['answer'][:300]}...")

In [ ]:
# Turn 2 - Follow-up (tests conversation memory)
result = rag.query("¿Cuál me recomiendas para ver el atardecer?")
print(f"👤 Usuario: {result['question']}")
print(f"🤖 Asistente: {result['answer']}")

In [ ]:
# Turn 3 - Weather for the recommended beach (tests function calling + memory)
result = rag.query("¿Qué tiempo hará mañana para ir allí?")
print(f"👤 Usuario: {result['question']}")
print(f"🤖 Asistente: {result['answer']}")
print(f"🔧 Tool called: {result.get('tool_called', False)}")

In [ ]:
# Test 5: Query that should NOT trigger weather
print("=" * 60)
print("TEST 5: Non-weather Query (should NOT trigger function)")
print("=" * 60)

rag.clear_history()
result = rag.query("¿Dónde puedo comer bien en Puerto de la Cruz?")
print(f"👤 Usuario: {result['question']}")
print(f"🤖 Asistente: {result['answer']}")
print(f"🔧 Tool called: {result.get('tool_called', False)}")

In [ ]:
# Show conversation history
print("📜 Historial de conversación:")
for i, msg in enumerate(rag.get_history()):
    role = "👤" if msg["role"] == "user" else "🤖"
    content = msg["content"][:60] + "..." if len(msg["content"]) > 60 else msg["content"]
    print(f"{i+1}. {role} {content}")